# default_exp python_call_graph

In [1]:
# export

import csrgraph as cg
import pandas as pd
import numpy as np
from github_search import python_tokens, paperswithcode_tasks
from mlutil.feature_extraction import embeddings
from mlutil import prototype_selection
import mlutil
from mlutil.feature_extraction import embeddings

import ast
import astunparse
import csrgraph
import nodevectors

import os
import itertools
import igraph
from sklearn import metrics
import gensim

from operator import itemgetter
import pickle

In [2]:
%cd ..

/home/kuba/Projects/github_search


In [4]:
%%time
python_files_df = pd.read_csv("data/crawled_python_files.csv", encoding="utf-8")

CPU times: user 46.2 s, sys: 2.96 s, total: 49.2 s
Wall time: 49.2 s


In [5]:
python_files_df.shape

(1402272, 3)

In [6]:
python_files_df.head()

,content,path,repo_name
0,import pickle\n\nimport tensorflow as tf\nimpo...,translate/import_graph.py,trangvu/ape-npi
1,import tensorflow as tf\nimport math\nfrom ten...,translate/models.py,trangvu/ape-npi
2,import tensorflow as tf\nfrom tensorflow.contr...,translate/conv_lstm.py,trangvu/ape-npi
3,import tensorflow as tf\nfrom translate import...,translate/beam_search.py,trangvu/ape-npi
4,from tensorflow.python.ops import init_ops\nfr...,translate/rnn.py,trangvu/ape-npi


In [7]:
%load_ext autoreload

In [3]:
code = """
def foo():
    bar()
"""

In [4]:
function_ast = ast.parse(code).body[0]

In [5]:
ast.parse(code).body[0].name

'foo'

In [6]:
exprs = ast.parse(code).body[0].body

In [7]:
expr = exprs[0]

In [8]:
expr_ast = ast.parse("word = tuple(orig) + ''").body[0]

In [9]:
expr_ast.value

In [93]:
#export


def get_calls_from_expr_or_assign(expr):
    try:
        if type(expr) is ast.Name:
            return []
        if type(expr) is ast.Call:
            # function call
            if type(expr.func) is ast.Name:
                name = [expr.func.id]
            # method
            elif type(expr.func) is ast.Attribute:
                name = [expr.func.attr]
            else:
                name = []
            return name + get_calls_from_expr_or_assign(expr.args)
        elif type(expr) is ast.Attribute:
            return [expr.attr]
        elif type(expr) is ast.BinOp:
            return get_calls_from_expr_or_assign(
                expr.left
            ) + get_calls_from_expr_or_assign(expr.right)
        elif type(expr) is ast.Expr:
            return get_calls_from_expr_or_assign(expr.value)
        elif type(expr) is ast.Assign:
            return get_calls_from_expr_or_assign(expr.value)
        elif type(expr) is ast.While or type(expr) is ast.If:
            return (
                get_calls_from_expr_or_assign(expr.test)
                + get_calls_from_expr_or_assign(expr.orelse)
                + get_calls_from_expr_or_assign(expr.body)
            )
        elif type(expr) is ast.For:
            return (
                get_calls_from_expr_or_assign(expr.target)
                + get_calls_from_expr_or_assign(expr.body)
                + get_calls_from_expr_or_assign(expr.iter)
            )
        elif type(expr) is ast.Try:
            return get_calls_from_expr_or_assign(expr.body)
        elif type(expr) is ast.If:
            return get_calls_from_expr_or_assign(
                expr.body
            ) + get_calls_from_expr_or_assign(expr.orelse)
        elif type(expr) is list:
            return [
                res
                for subexpr in expr
                for res in get_calls_from_expr_or_assign(subexpr)
            ]
        else:
            return []
    except Exception as e:
        logging.info(str(astunparse.unparse(expr)), " ", str(e))
        return []



In [94]:
encode_code = """def encode(self, orig):
    word = tuple(orig) + ('</w>',)
    pairs = get_pairs(word)

    while True:
        bigram = min(pairs, key=lambda pair: self.bpe_codes.get(pair, float('inf')))
        if bigram not in self.bpe_codes:
            break
        first, second = bigram
        new_word = []
        i = 0
        while i < len(word):
            try:
                j = word.index(first, i)
                new_word.extend(word[i:j])
                i = j
            except:
                new_word.extend(word[i:])
                break

            if word[i] == first and i < len(word) - 1 and word[i + 1] == second:
                new_word.append(first + second)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        new_word = tuple(new_word)
        word = new_word
        if len(word) == 1:
            break
        else:
            pairs = get_pairs(word)

    # don't print end-of-word symbols
    if word[-1] == '</w>':
        word = word[:-1]
    elif word[-1].endswith('</w>'):
        word = word[:-1] + (word[-1].replace('</w>', ''),)

    return word
"""

In [95]:
encode_ast = ast.parse(encode_code)

In [96]:
assign_body = encode_ast.body[0].body[0]

In [97]:
class_def = ast.parse(code).body[0]

In [98]:
encode_ast

In [99]:
# export


def zip_dicts(dicts):
    res = {}
    for d in dicts:
        res = {**res, **d}
    return res


def get_ast_function_calls(code_ast, calls={}):
    if type(code_ast) is ast.FunctionDef:
        return {
            code_ast.name: frozenset(
                sum([get_calls_from_expr_or_assign(expr) for expr in code_ast.body], [])
            )
        }
    elif type(code_ast) is ast.ClassDef:
        return {
            code_ast.name: frozenset(
                sum([get_calls_from_expr_or_assign(expr) for expr in code_ast.body], [])
            )
        }
    elif hasattr(code_ast, "body"):
        return zip_dicts([get_ast_function_calls(item) for item in code_ast.body])
    else:
        return {}


def get_function_calls(code):
    code_ast = ast.parse(code)
    return get_ast_function_calls(code_ast)

In [100]:
e = ast.parse("new_word.extend(word[i:j])").body[0]

In [101]:
get_calls_from_expr_or_assign(ast.parse("word[i:j]").body[0])

[]

In [102]:
raw_exp_text = "def foo(x): f(g(x))"
raw_exp = ast.parse("def foo(x): f(g(x))")

In [103]:
get_ast_function_calls(encode_ast.body[0])

{'encode': frozenset({'append',
            'endswith',
            'extend',
            'get_pairs',
            'index',
            'min',
            'tuple'})}

In [90]:
get_ast_function_calls(encode_ast.body[0])

{'encode': frozenset({'append',
            'bigram',
            'endswith',
            'extend',
            'first',
            'get_pairs',
            'i',
            'index',
            'j',
            'min',
            'new_word',
            'orig',
            'pairs',
            'second',
            'tuple',
            'word'})}

In [91]:
get_function_calls(raw_exp_text)

{'foo': frozenset({'f', 'g', 'x'})}

In [85]:
[deps] = get_ast_function_calls(raw_exp).values()
assert deps == frozenset(["f", "g"])

AssertionError: 

In [84]:
deps

frozenset({'f', 'g', 'x'})

In [67]:
for item in raw_exp.body:
    print(astunparse.unparse(item), type(item))



def foo(x):
    f(g(x))
 <class '_ast.FunctionDef'>


In [68]:
get_function_calls(code)

{'foo': frozenset({'bar'})}

In [69]:
assert get_function_calls(code) == {"foo": frozenset(["bar"])}

In [70]:
items = ast.parse(code).body

In [28]:
example_file = python_files_df["content"].iloc[1]
print(example_file)

import tensorflow as tf
import math
from tensorflow.contrib.rnn import BasicLSTMCell, RNNCell, DropoutWrapper, MultiRNNCell
from translate.rnn import stack_bidirectional_dynamic_rnn, CellInitializer, GRUCell, DropoutGRUCell, PLSTM
from translate.rnn import get_state_size
from translate.beam_search import get_weights
from translate import utils, beam_search
from translate.conv_lstm import BasicConvLSTMCell


def auto_reuse(fun):
    """
    Wrapper that automatically handles the `reuse' parameter.
    This is rather risky, as it can lead to reusing variables
    by mistake.
    """

    def fun_(*args, **kwargs):
        try:
            return fun(*args, **kwargs)
        except ValueError as e:
            if 'reuse' in str(e):
                with tf.variable_scope(tf.get_variable_scope(), reuse=True):
                    return fun(*args, **kwargs)
            else:
                raise e

    return fun_


get_variable = auto_reuse(tf.get_variable)
dense = auto_reuse(tf.layers.den

In [29]:
fn_code = ast.parse(example_file).body[-2]
print(astunparse.unparse(fn_code))



def reinforce_baseline(decoder_states, reward):
    '\n    Center the reward by computing a baseline reward over decoder states.\n\n    :param decoder_states: internal states of the decoder, tensor of shape (batch_size, time_steps, state_size)\n    :param reward: reward for each time step, tensor of shape (batch_size, time_steps)\n    :return: reward - computed baseline, tensor of shape (batch_size, time_steps)\n    '
    baseline = dense(tf.stop_gradient(decoder_states), units=1, activation=None, name='reward_baseline', kernel_initializer=tf.constant_initializer(0.01))
    baseline = tf.squeeze(baseline, axis=2)
    return (reward - baseline)



In [30]:
get_function_calls(example_file)

{'auto_reuse': frozenset(),
 'CellWrapper': frozenset(),
 'multi_encoder': frozenset({'cell_size',
            'cell_type',
            'concat',
            'encoder_states',
            'encoders',
            'enumerate',
            'get_state_size',
            'lstm_proj_size',
            'random_normal_initializer',
            'random_uniform_initializer'}),
 'compute_energy': frozenset({'attn_filters',
            'attn_size',
            'conv',
            'conv2d',
            'dense',
            'dropout',
            'expand_dims',
            'filter_',
            'filter_shape',
            'get_variable',
            'hidden',
            'input_length',
            'layer_norm',
            'log',
            'mult_attn',
            'pos_feats',
            'prev_weights',
            'range',
            'reshape',
            'squeeze',
            'stack',
            'state',
            'tile',
            'time',
            'time_steps',
            'to_flo

In [31]:
items = ast.parse("a.append(1)").body
expr = items[0].value

In [32]:
# export


def get_sample_files_df(python_files_df, n_files=100, repo_col="repo_name"):
    "sample n_files from each repo"
    if n_files == None:
        n_files = python_files_df.shape[0]
    return (
        python_files_df.dropna()
        .groupby(repo_col)
        .apply(lambda df: df.iloc[: min(df.shape[0], n_files)])
        .reset_index(drop=True)
    )

In [33]:
sample_df = (
    python_files_df.dropna()
    .groupby("repo_name")
    .apply(lambda df: df.iloc[: min(df.shape[0], 100)])
    .reset_index(drop=True)
)

In [34]:
sample_df.columns

Index(['content', 'path', 'repo_name'], dtype='object')

In [35]:
# export


def clean_task_names(tasks):
    return (
        tasks.str.replace("2D ", "")
        .str.replace("3D ", "")
        .str.replace("4D ", "")
        .str.replace("6D ", "")
    )


def get_repo_task_edges(python_files_df):
    task_exploded_df = python_files_df[["repo_name", "tasks"]].explode("tasks")
    task_exploded_df["tasks"] = clean_task_names(task_exploded_df["tasks"])
    return (
        task_exploded_df.groupby("tasks")
        .apply(lambda df: {df["tasks"].iloc[0]: frozenset(df["repo_name"].values)})
        .tolist()
    )

In [36]:
sample_df.columns

Index(['content', 'path', 'repo_name'], dtype='object')

In [37]:
# export


def try_run(f):
    def _maybe_failed_f(args):
        try:
            return f(args)
        except:
            return None

    return _maybe_failed_f

In [38]:
# export
import tqdm


def get_upper_level_edges(upper_level_names, lower_level_edges):
    return [
        {
            file_name: frozenset(
                [encode_bytes(function) for function in lower_level_edges.keys()]
            )
        }
        for (file_name, lower_level_edges) in zip(upper_level_names, lower_level_edges)
        if type(lower_level_edges) is dict and len(lower_level_edges.keys()) > 0
    ]


def make_records(function_edges):
    return [
        {"calling_function": calling_fn, "called_function": called_fn}
        for calling_fn in function_edges.keys()
        for called_fn in function_edges[calling_fn]
    ]

In [143]:
# export


def encode_bytes(s):
    return bytes(s, "UTF-8")


class RepoDependencyGraphFetcher:
    def clean_content(self, python_files_df, cleaned_column_name="clean_content"):
        python_files_df[cleaned_column_name] = python_files_df["content"].str.replace(
            "429: Too Many Requests", ""
        )
        python_files_df[cleaned_column_name] = python_files_df["clean_content"].apply(
            lambda s: np.nan if s == "" else s
        )
        return python_files_df

    def get_repo_and_root_edges(
        self, python_files_df, path_col="path", add_repo_label=False
    ):
        filenames = (
            python_files_df[path_col].apply(os.path.basename).str.replace(".py", "")
        )

        if add_repo_label:
            filenames = python_files_df["repo_name"] + ":" + filenames
        filenames = filenames.apply(encode_bytes)
        repo_edges_dict = (
            filenames.groupby(python_files_df["repo_name"])
            .agg(lambda args: frozenset(args))
            .to_dict()
        )
        repo_edges = [{k: repo_edges_dict[k]} for k in repo_edges_dict]
        root_edges = [
            {
                "<ROOT>": frozenset(
                    repo for edges in repo_edges for repo in repo_edges_dict.keys()
                )
            }
        ]
        return root_edges, repo_edges

    def get_filename_and_function_edges(
        self, python_files_df, clean_content, add_repo_label=False, path_col="path"
    ):
        if clean_content:
            content_column = "clean_content"
            python_files_df = self.clean_content(python_files_df, content_column)
        else:
            content_column = "content"

        filenames = (
            python_files_df[path_col].apply(os.path.basename).str.replace(".py", "")
        )
        if add_repo_label:
            filenames = python_files_df["repo_name"] + ":" + filenames
        filenames = filenames.apply(encode_bytes)
        files = python_files_df[content_column]
        function_edges = files.apply(try_run(get_function_calls)).dropna()
        function_edges = function_edges.to_list()
        filename_edges = get_upper_level_edges(filenames, function_edges)
        return filename_edges, function_edges

    def get_records(self, edges):
        return [record for edge_group in edges for record in make_records(edge_group)]

    def make_dependency_df(self, records, edge_type):
        dependency_records_df = pd.DataFrame.from_records(records)
        dependency_records_df.columns = ["source", "destination"]
        dependency_records_df["edge_type"] = edge_type
        dependency_records_df = dependency_records_df[
            (dependency_records_df["source"].apply(len) > 0)
            & (dependency_records_df["source"] != "null")
            & (dependency_records_df["destination"].apply(len) > 0)
            & (dependency_records_df["source"] != dependency_records_df["destination"])
        ]
        return dependency_records_df.drop_duplicates()

    def get_dependency_df(
        self,
        python_files_df,
        dep_type="repo",
        add_filename_repo_label=False,
        clean_content=True,
    ):
        print("constructing edges")
        if dep_type == "repo":
            root_edges, repo_edges = self.get_repo_and_root_edges(
                python_files_df, add_repo_label=add_filename_repo_label
            )
            edge_groups = [root_edges, repo_edges]
            record_groups = [self.get_records(edge_group) for edge_group in edge_groups]
            edge_types = ["root-repo", "repo-file"]
        elif dep_type == "function":
            filename_edges, function_edges = self.get_filename_and_function_edges(
                python_files_df, clean_content, add_repo_label=add_filename_repo_label
            )
            edge_groups = [filename_edges, function_edges]
            record_groups = [self.get_records(edge_group) for edge_group in edge_groups]
            edge_types = ["file-function", "function-function"]
        else:
            raise NotImplementedError("unsupported dep_type: {}".format(dep_type))
        print("creating dependency dataframe")
        return pd.concat(
            [
                self.make_dependency_df(records, edge_type)
                for edge_type, records in zip(edge_types, record_groups)
            ]
        ).drop_duplicates(subset=["source", "destination"], keep="first")

    def prepare_dependency_records(
        self, python_files_df, add_filename_repo_label=False
    ):
        repo_records_df = self.get_dependency_df(
            python_files_df,
            "repo",
            clean_content=True,
            add_filename_repo_label=add_filename_repo_label,
        )
        function_records_df = self.get_dependency_df(
            python_files_df,
            "function",
            clean_content=True,
            add_filename_repo_label=add_filename_repo_label,
        )
        dependency_records_df = pd.concat([repo_records_df, function_records_df])
        dependency_records_df["source"] = dependency_records_df["source"].apply(
            lambda s: s if type(s) is str else s.decode("utf-8")
        )
        dependency_records_df["destination"] = dependency_records_df[
            "destination"
        ].apply(lambda s: s if type(s) is str else s.decode("utf-8"))
        return dependency_records_df

In [144]:
# export


def get_node_degrees(records_df):
    outdegree = records_df["source"].value_counts()
    indegree = records_df["destination"].value_counts()
    degree = outdegree.add(indegree, fill_value=0)
    return outdegree, indegree, degree

In [ ]:
# export


def get_aggregated_edge_type_df(records_df, edge_type):
    sample_df = records_df[records_df["edge_type"] == edge_type]
    repos = sample_df.groupby("source").groups.keys()
    descriptions = sample_df.groupby("source").apply(
        lambda df: " ".join(set(df["destination"].sample(min(len(df), 1000))))
    )
    raw_descriptions_df = descriptions.reset_index()
    descriptions_df = raw_descriptions_df.apply(
        lambda item: item["source"] + " " + item[0].replace(item["source"] + ":", ""),
        axis=1,
    )
    descriptions_df.index = raw_descriptions_df["source"]
    return descriptions_df


def get_descriptions(records_df):
    repo_descriptions = get_aggregated_edge_type_df(records_df, "repo-file")
    file_descriptions = get_aggregated_edge_type_df(records_df, "file-function")
    file_descriptions.name = "file_description"
    repo_descriptions.name = "repo_description"
    return repo_descriptions, file_descriptions


def get_description_records_df(records_df):
    repo_descriptions, file_descriptions = get_descriptions(records_df)
    return records_df.merge(
        repo_descriptions, left_on="source", right_index=True
    ).merge(file_descriptions, left_on="destination", right_index=True)

In [145]:
sample_files = python_files_df.iloc[:1000]

In [146]:
wrapper = RepoDependencyGraphFetcher()

In [147]:
sample_files

,content,path,repo_name
0,import pickle\n\nimport tensorflow as tf\nimpo...,translate/import_graph.py,trangvu/ape-npi
1,import tensorflow as tf\nimport math\nfrom ten...,translate/models.py,trangvu/ape-npi
2,import tensorflow as tf\nfrom tensorflow.contr...,translate/conv_lstm.py,trangvu/ape-npi
3,import tensorflow as tf\nfrom translate import...,translate/beam_search.py,trangvu/ape-npi
4,from tensorflow.python.ops import init_ops\nfr...,translate/rnn.py,trangvu/ape-npi
...,...,...,...
995,"# flake8: noqa\n# There's no way to ignore ""F4...",src/transformers/models/byt5/__init__.py,huggingface/transformers
996,"# coding=utf-8\n# Copyright 2019-present, the ...",src/transformers/models/retribert/configuratio...,huggingface/transformers
997,"# flake8: noqa\n# There's no way to ignore ""F4...",src/transformers/models/retribert/__init__.py,huggingface/transformers
998,# coding=utf-8\n# Copyright 2018 The HuggingFa...,src/transformers/models/retribert/tokenization...,huggingface/transformers


In [162]:
records_df = wrapper.prepare_dependency_records(
    sample_files, add_filename_repo_label=True
)

constructing edges
creating dependency dataframe
constructing edges


<ipython-input-143-3a361432cfe4>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  filenames = python_files_df[path_col].apply(os.path.basename).str.replace('.py', '')
<ipython-input-143-3a361432cfe4>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  python_files_df[cleaned_column_name] = python_files_df['content'].str.replace("429: Too Many Requests", "")
<ipython-input-143-3a361432cfe4>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  python_files_df[clea

creating dependency dataframe


In [163]:
labeled_file_destinations = records_df[records_df["edge_type"] == "repo-file"][
    "destination"
]
assert "harryhan618/LaneNet:postprocess" in labeled_file_destinations.values

In [164]:
records_df = wrapper.prepare_dependency_records(
    sample_files, add_filename_repo_label=False
)

constructing edges
creating dependency dataframe
constructing edges


<ipython-input-143-3a361432cfe4>:15: FutureWarning: The default value of regex will change from True to False in a future version.
  filenames = python_files_df[path_col].apply(os.path.basename).str.replace('.py', '')
<ipython-input-143-3a361432cfe4>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  python_files_df[cleaned_column_name] = python_files_df['content'].str.replace("429: Too Many Requests", "")
<ipython-input-143-3a361432cfe4>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  python_files_df[clea

creating dependency dataframe


In [165]:
labeled_file_destinations = records_df[records_df["edge_type"] == "repo-file"][
    "destination"
]
assert "postprocess" in labeled_file_destinations.values

In [ ]:
# export


def get_records_df_with_labeled_files(records_df):
    outdegree, indegree, degree = get_node_degrees(records_df)
    labeled_files_dependency_records_df = records_df.copy()
    labeled_files = labeled_files_dependency_records_df[
        labeled_files_dependency_records_df["edge_type"] == "repo-file"
    ]["destination"]
    labeled_files = (
        labeled_files_dependency_records_df[
            labeled_files_dependency_records_df["edge_type"] == "repo-file"
        ]["source"]
        + ":"
        + labeled_files
    )
    labeled_files_dependency_records_df["destination"][
        labeled_files_dependency_records_df["edge_type"] == "repo-file"
    ] = labeled_files
    return labeled_files_dependency_records_df

In [ ]:
dependency_records_df = pd.read_csv("output/dependency_records.csv")

In [ ]:
outdegree, indegree, degree = get_node_degrees(dependency_records_df)

In [ ]:
degree

In [167]:
"trangvu/ape-npi" in python_files_df["repo_name"].unique()

True

In [ ]:
%%time
dependency_records_df = repo_dependency_graph_fetcher.get_dependency_df(python_files_df)

In [ ]:
def encode_bytes(s):
    return bytes(s, "UTF-8")

In [ ]:
bytes("a", "UTF-8")

In [ ]:
b"a".decode("UTF-8")

In [ ]:
dependency_records_df.to_csv("data/raw_dependency_records.csv", index=False)

In [ ]:
total_counts = pd.concat(
    [dependency_records_df["source"], dependency_records_df["destination"]]
).value_counts()
valid_nodes = total_counts[(total_counts > 1)].index
valid_dependency_records_df = dependency_records_df[
    dependency_records_df["source"].isin(valid_nodes)
    & dependency_records_df["destination"].isin(valid_nodes)
]
valid_total_counts = pd.concat(
    [valid_dependency_records_df["source"], valid_dependency_records_df["destination"]]
).value_counts()

In [ ]:
dependency_records_df.info()

In [ ]:
dependency_records_df["edge_type"].value_counts()

In [ ]:
repo_edges = dependency_records_df["source"][
    dependency_records_df["edge_type"] == "repo-file"
]

In [ ]:
dependency_records_df.shape

# Exploration

In [ ]:
dependency_records_df[dependency_records_df["source"] == "fairseq"]

In [ ]:
dependency_records_df[dependency_records_df["source"] == "wgan"]

In [ ]:
dependency_records_df[dependency_records_df["source"] == "LinearExplainer"]

In [ ]:
dependency_records_df["destination"].value_counts().describe()  # plot.hist()

In [ ]:
dependency_records_df.info()

In [ ]:
dependency_records_df["edge_type"].value_counts()

In [ ]:
# dependency_records_df = pd.read_csv("data/raw_dependency_records.csv")

In [ ]:
dependency_records_df.to_csv("data/raw_dependency_records.csv", index=False)

In [ ]:
dependency_records_df

In [ ]:
valid_dependency_records_df.to_csv("data/dependency_records.csv", index=False)

In [ ]:
valid_dependency_records_df.info()

In [ ]:
valid_dependency_records_df[valid_dependency_records_df["edge_type"] == "repo-file"]

In [ ]:
self = repo_dependency_graph_fetcher

In [ ]:
root_edges, repo_edges = self.get_repo_and_root_edges(sample_files_df)

In [ ]:
edge_groups = [root_edges, repo_edges]

In [ ]:
len(root_edges[0]["<ROOT>"])

In [ ]:
record_groups = [self.get_records(edge_group) for edge_group in edge_groups]

In [ ]:
function_records_df.info()

In [ ]:
repo_records_df

In [ ]:
sample_files_df = get_sample_files_df(python_files_df, n_files=1000)
repo_records_df = repo_dependency_fetcher.get_dependency_df(
    sample_files_df, "repo", clean_content=True
)
function_records_df = repo_dependency_fetcher.get_dependency_df(
    sample_files_df, "function", clean_content=True
)
dependency_records_df = pd.concat([repo_records_df, function_records_df])
dependency_records_df["source"] = dependency_records_df["source"].apply(
    lambda s: s if type(s) is str else s.decode("utf-8")
)
dependency_records_df["destination"] = dependency_records_df["destination"].apply(
    lambda s: s if type(s) is str else s.decode("utf-8")
)

In [ ]:
b"1".decode("utf-8")

In [ ]:
dependency_records_df["destination"] = dependency_records_df["destination"].apply(
    lambda s: s if type(s) is str else s.decode("utf-8")
)

In [ ]:
dependency_records_df.to_csv("data/dependency_records.csv", index=False)